In [189]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import pickle
from pathlib import Path
import os
import tqdm


EPSILON = 1e-3

In [190]:
class City:

    # CONSTRUCTOR
    def __init__(self, housing, amts, rho=2): #default rho (house capacity) == 2

        self.rho = rho #house capacity
        self.amts = amts #amenity list
        self.housing = housing #housing list
        
        # Create dictionary containing ID and tuple from 'housing' list
        self.housingDict = {} 
        for ID, (lat, lon, name, beltline) in enumerate(self.housing): #Iterate through each housing node
            self.housingDict[ID] = {
                'lat': lat, # Latitude
                'lon': lon, # Longitude
                'name': name, # Housing name (region)
                'beltline': beltline,  # Is it in the Beltline?
                
                'amt': False,  # Is it an amenity?
                'inh': set(),  # Set containing all Agent inhabitants
                'dow_thr': 0.0,  # Endowment threshold initialized to 0
                'upk': False,  # Upkeep score
                'cmt': 0.0,  # Community score
                'pop_hist': [], # Population history
                'cmt_hist': []  # Community history
            }
            
        #Create dictionary containing ID and tuple from 'amts' list
        self.amtsDict = {}
        for ID, (lat, lon, name, beltline) in enumerate(self.amts): #Iterate through each housing node
            self.amtsDict[ID+len(self.housing)] = {
                'lat': lat, # Latitude
                'lon': lon, # Longitude
                'name': name, # Name
                'beltline': beltline,  # Is it in the Beltline?
                
                'amt': True,  # Is it an amenity?

                'inh': None,  # Set containing all Agent inhabitants
                'dow_thr': None, # Endowment threshold initialized to 0
                'upk': None,  # Upkeep score
                'cmt': 0.0,  # Community score
                'pop_hist': None,  # Population history
                'cmt_hist': None  # Community history
            }

        # Set transit amenities [GRAPH APPROACH]
        '''def set_amts(self, amts):
        self.amts = amts
        for u in self.amts:
            data = self.housingDict[u]
            
            # Node characteristics of an amenity node
            data['amt'] = True
            data['beltline'] = self.amts[u][3]
            data['inh'] = None
            data['dow_thr'] = None
            data['upk'] = None
            data['cmt'] = None
            data['pop_hist'] = None
            data['cmt_hist'] = None
            
        # Calculate shortest distance between each housing node & transit node 
        # NECESSARY FOR ACCESSIBILITY SCORE
        if len(self.amts) == 0:
            self.amts_dist = None
        else:
            self.amts_dist = np.array([self.calcDistance(node1, node2) for node1 in self.housing for node2 in self.amts])'''
        
        # Dictionary containing all nodes
        self.allNodes = {**self.housingDict, **self.amtsDict}

        # Fill self.housing_dist [GRAPH APPROACH] 
        ''' 
        self.diam = nx.diameter(self.g, weight='length') #Longest shortest path between any two nodes on graph
        
        #Normalized shortest paths between all pairs of nodes
        self.housing_dist = dict(nx.all_pairs_dijkstra_path_length(self.g, weight='length'))
        self.housing_dist = pd.DataFrame.from_dict(self.housing_dist).sort_index() / self.diam #
        self.housing_dist = self.housing_dist.to_numpy()'''

        self.housing_dist = {} # Dictionary of distances btw housing node pairs   
        if len(self.housingDict) != 0:
            self.get_housing_distances() # Fill self.housing_dist dictionary

        self.amts_dist = {} # Dictionary of distances btw housing and amenity 
        if len(self.amtsDict) != 0:
            self.get_amt_distances() # Fill self.housing_dist dictionary

        self.agts = None #List of agents
        self.agt_dows = None #List of agent endowments


    # HELPER FUNCTION - fill self.housing_dist dictionary
    def get_housing_distances(self):
        numHousing = len(self.housingDict)
        self.housing_dist = np.zeros((numHousing, numHousing))
        
        for ID1, node1 in self.housingDict.items():
            for ID2, node2 in self.housingDict.items():
                if ID1 <= ID2:  
                    # Indices [ID1][ID2] & [ID2][ID1] have the same distance
                    distance = self.calcDistance(node1, node2)
                    self.housing_dist[ID1][ID2] = distance
                    self.housing_dist[ID2][ID1] = distance

        #normalize with longest shortest distance
        if np.max(self.housing_dist) != 0:
            self.housing_dist = self.housing_dist / np.max(self.housing_dist)
        else:
            self.housing_dist = np.zeros_like(self.housing_dist)
    
    # HELPER FUNCTION - fill self.amts_dist dictionary
    def get_amt_distances(self):
        numHousing = len(self.housingDict)
        numAmts = len(self.amtsDict)
        self.amts_dist = np.zeros((numHousing, numAmts))
        
        for ID1, node1 in self.housingDict.items():
            for ID2, node2 in self.amtsDict.items():
                distance = self.calcDistance(node1, node2)
                self.amts_dist[ID1][ID2 - len(self.housing)] = distance

        #normalize with longest shortest distance
        if self.amts_dist.size != 0:
            self.amts_dist = self.amts_dist / np.max(self.amts_dist) 

    # HELPER FUNCTION - calculates distance between ANY two nodes
    def calcDistance(self, node1, node2):
        # PROCESS FOR CALCULATING (HAVERSINE) GEOGRAPHIC DISTANCES
        R = 6371.0 # Earth radius (km)
        lat1 = np.radians(node1['lat'])
        lon1 = np.radians(node1['lon'])
        lat2 = np.radians(node2['lat'])
        lon2 = np.radians(node2['lon'])

        # Haversine formula
        a = np.sin((lat2 - lat1) / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        distance = R * c
        return distance

    def set_agts(self, agts):
        self.agts = agts #list of agents
        self.agt_dows = np.array([a.dow for a in self.agts]) #array of agent endowments

    # Update each node
    def update(self):   
        for ID, data in self.housingDict.items(): # Iterate through dict
            # Skip amenity nodes
            if data['amt']:
                continue

            pop = len(data['inh']) # Population
            inhabitant_dows = [a.dow for a in data['inh']]  # Array of endowments of node's inhabitants
            
            # COMMUNITY SCORE (average endowment)
            cmt = 0.0 #TODO: review logic: set to 0 if population is 0?
            if pop > 0:
                #average endowment of agents in set data['inh'], weighted by alpha
                cmt = np.average(inhabitant_dows, weights=[(1 - self.housing_dist[ID][a.u]) ** 2 for a in data['inh']]) # Community value of node
                
                data['cmt'] = cmt
            
            # UPKEEP SCORE
            # ENDOWMENT THRESHOLD
            if pop > 0: # If inhabited
                if pop < self.rho:
                    data['dow_thr'] = 0.0
                else:
                    data['dow_thr'] = sorted([a.dow for a in data['inh']])[-self.rho] # Lowest endowment value if Population = Rho
                data['upk'] = True
                
            else: # If uninhabited
                data['dow_thr'] = 0.0
                data['upk'] = False

            # Update population history
            data['pop_hist'].append(pop)
            
            # Update Community history (average endowment)
            data['cmt_hist'].append(cmt)
        

#[GRAPH APPROACH]
# 'plot' function
'''
    def plot(self, cmap='YlOrRd', figkey=None):

        for u, data in self.g.nodes(data=True):
            if not data['amt']: # Non-transit nodes
                data['dow'] = np.average(self.agt_dows, weights=[a.avg_probabilities[u] for a in self.agts]) # Average endowment
                data['dow'] = (data['dow'] - min(city.agt_dows)) / (max(city.agt_dows) - min(city.agt_dows)) # Normalize
                
                data['pop'] = np.sum([a.avg_probabilities[u] for a in self.agts]) # Sum agent probabilities to be at the node
            else: # Transit nodes
                data['dow'] = np.nan
                data['pop'] = np.nan

        # ===============================
        # SIMULATION VISUALIZATION - CODE
        # ===============================
        no_agts = len(self.agts) # Number of agents
        
        
        # Size of nodes
        node_size = [no_agts / 10 * data['pop'] if not data['amt'] else no_agts / 2.5 for _, data in self.g.nodes(data=True)]
        
        # Color of nodes
        node_color = ox.plot.get_node_colors_by_attr(self.g, 'dow', start=0, stop=1, na_color='b', cmap=cmap)
        
        fig, ax = plt.subplots(figsize=(9, 6))

        # COLORBAR
        cb = fig.colorbar(
            plt.cm.ScalarMappable(cmap=plt.colormaps[cmap]), ax=ax, location='bottom', shrink=0.5, pad=0.05
        )
        cb.set_label('Expected Endowment', fontsize=14)
        
        # PLOT GRAPH
        ox.plot_graph(self.g, ax=ax, bgcolor='w', node_color=node_color, node_size=node_size)
        plt.show()
        
        if figkey is not None:
            plt.savefig('./figures/{0}.pdf'.format(figkey), bbox_inches='tight', format='pdf')
        # Save graph as pdf in './figures/' directory if 'figkey' is provided
        '''

"\n    def plot(self, cmap='YlOrRd', figkey=None):\n\n        for u, data in self.g.nodes(data=True):\n            if not data['amt']: # Non-transit nodes\n                data['dow'] = np.average(self.agt_dows, weights=[a.avg_probabilities[u] for a in self.agts]) # Average endowment\n                data['dow'] = (data['dow'] - min(city.agt_dows)) / (max(city.agt_dows) - min(city.agt_dows)) # Normalize\n                \n                data['pop'] = np.sum([a.avg_probabilities[u] for a in self.agts]) # Sum agent probabilities to be at the node\n            else: # Transit nodes\n                data['dow'] = np.nan\n                data['pop'] = np.nan\n\n        # ===============================\n        # SIMULATION VISUALIZATION - CODE\n        # ===============================\n        no_agts = len(self.agts) # Number of agents\n        \n        \n        # Size of nodes\n        node_size = [no_agts / 10 * data['pop'] if not data['amt'] else no_agts / 2.5 for _, data in self.g

In [191]:
class Agent:
    
    def __init__(self, i, dow, city, alpha=0.5):

        self.i = i # Agent instance identifier
        self.dow = dow #endowment
        self.city = city #city
        self.alpha = alpha #Transit_access/Community weight

        self.weights = [1] * len(self.city.housingDict)
        self.probabilities = [1] * len(self.city.housingDict) # Each element corresponds to a different node
        self.tot_probabilities = 0.0 # Total probabilities of going to each node, added up
        self.avg_probabilities = None # Used for graphing purposes
        self.u = None

        self.reset()
        
    # Create hash identifier
    def __hash__(self):
        return hash(self.i)

    def __eq__(self, other): 
        return self.i == other.i

# RESET METHOD
# Starting point at beginning of simulation
    def reset(self):
        # Normalize probabilities
        self.probabilities[:] = [a/len(self.probabilities) for a in self.probabilities]
        
        self.tot_probabilities = np.sum(self.probabilities)
        
        # Current node - Initialize starting position at random node (based on weights)
        self.u = np.random.choice(list(self.city.housingDict.keys()), p=self.probabilities) 
        
        # Adds self to node
        self.city.housingDict[self.u]['inh'].add(self)

# ACTION METHOD
    def act(self): 
        # Leave node
        self.city.housingDict[self.u]['inh'].remove(self) 
    
        # Choose another node
        self.u = np.random.choice(list(self.city.housingDict.keys()), p=self.probabilities) 
    
        # Join node
        self.city.housingDict[self.u]['inh'].add(self) 
        
# LEARN METHOD
    def learn(self):
        for ID, _ in self.city.housingDict.items(): # for each node
            self.weights[ID] *= (1 - EPSILON * self.cost(ID)) # adjust probability weights based off Cost Function
        
        self.probabilities = np.array(self.weights / np.sum(self.weights)) # (normalize) self.probabilities; each element is the probability of going to that node
        
        #TODO: Set self.tot_probabilities to 0 before adding up new self.probabilities?
        for a in self.probabilities:
            self.tot_probabilities += a # used for averaging purposes

# COST FUNCTION
    def cost(self, ID):
        
        # AFFORDABILITY SCORE
        # 1 if self.dow >= node.dow_thr; else 0
        aff = int(self.dow >= self.city.housingDict[ID]['dow_thr'])
        
        # UPKEEP SCORE
        # 1 if upkeep == True; else 0
        upk = int(self.city.housingDict[ID]['upk'])
        
        # BELTLINE SCORE
        # 1 if in beltline; else 0
        beltline = int(self.city.housingDict[ID]['beltline'])
    
        # DISTANCE SCORE
        # 1 if no amenities, 
        if  not self.city.amtsDict: 
            loc = 1 
        else:
            loc = np.exp(- (1 - self.alpha) * self.city.amts_dist[ID])
        
        # COMMUNITY SCORE
        # Difference between node 'cmt' value and self.dow.
        cmt = np.exp(- self.alpha * np.abs(self.dow - self.city.housingDict[ID]['cmt']))
    
        # COST FUNCTION
        c = 1 - aff * loc * beltline * upk * cmt
        
        return c

In [192]:
import requests
import zipfile
from pathlib import Path
from tqdm import tqdm

def download_and_extract_file(url, filename):
    # Use a 'data' subfolder in the current working directory
    cwd = Path.cwd()
    data_dir = cwd / "data"
    data_dir.mkdir(exist_ok=True)  # Create the 'data' directory if it doesn't exist
    file_path = data_dir / filename

    # Create extraction subfolder name (remove .zip extension)
    extract_folder_name = filename.rsplit('.', 1)[0]
    extract_path = data_dir / extract_folder_name
    
    # Check if file already exists
    if file_path.exists():
        print(f"{filename} already exists in {data_dir}. Skipping download.")
    else:
        # Make the request
        print(f"Downloading {filename} to {data_dir}...")
        response = requests.get(url, stream=True)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Get the total file size
            total_size = int(response.headers.get('content-length', 0))

            # Open the file and use tqdm for the progress bar
            with file_path.open('wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as progress_bar:
                for data in response.iter_content(chunk_size=1024):
                    size = file.write(data)
                    progress_bar.update(size)
            print(f"Successfully downloaded {filename}")
        else:
            print(f"Failed to download {filename}. Status code: {response.status_code}")
            return

    # Extract the ZIP file
    print(f"Extracting {filename} to {extract_path}...")
    extract_path.mkdir(exist_ok=True)  # Create the extraction folder if it doesn't exist
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        # Get the total number of files in the ZIP
        total_files = len(zip_ref.infolist())
        
        # Use tqdm for the extraction progress bar
        for file in tqdm(zip_ref.infolist(), desc="Extracting", total=total_files):
            zip_ref.extract(file, extract_path)
    
    print(f"Successfully extracted {filename} to {extract_path}")

# URL of the file to download
url = "https://www2.census.gov/geo/tiger/TIGER2022/ZCTA520/tl_2022_us_zcta520.zip"

# Filename to save as
filename = "tl_2022_us_zcta520.zip"

# Call the function to download and extract the file
download_and_extract_file(url, filename)

tl_2022_us_zcta520.zip already exists in c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data. Skipping download.
Extracting tl_2022_us_zcta520.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\tl_2022_us_zcta520...


Extracting: 100%|██████████| 7/7 [00:04<00:00,  1.75it/s]

Successfully extracted tl_2022_us_zcta520.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\tl_2022_us_zcta520


In [193]:
cwd = Path.cwd()
print(cwd)
figures_folder = Path(cwd / "figures")
if not os.path.isdir(figures_folder):
    os.makedirs(figures_folder)


# Housing amenities [manual]
# Use dummy coordinates for now
housing = [
    (0.0, 0.0, 'RDA/Cascade', True),
    (0.0, 0.0, 'Pittsburgh/Peoplestown', True),
    (0.0, 0.0, 'Boulevard Crossing', True),
    (0.0, 0.0, 'Memorial Drive/Glenwood', True),
    (0.0, 0.0, 'Freedom Parkway', True),
    (0.0, 0.0, 'Virginia Highlands/Ansley', True),
    (0.0, 0.0, 'Peachtree/Collier', True),
    (0.0, 0.0, 'Upper Westside/Northside', True),
    (0.0, 0.0, 'Simpson/Hollowell', True),
    (0.0, 0.0, 'Upper Marietta/Westside Park', True),

    (0.0, 0.0, 'Westlake HS', False), #4280
    (0.0, 0.0, 'Wheeler HS', False), #4292
    (0.0, 0.0, 'Lakeside HS', False), #4294
    (0.0, 0.0, 'Druid Hills HS', False), #4307
    (0.0, 0.0, 'McNair, Ronald E. HS', False), #4315
    (0.0, 0.0, 'Brookhaven City', False), #4387
    (0.0, 0.0, 'Decatur City', False), #4444
    (0.0, 0.0, 'East Point City', False), #4459
    (0.0, 0.0, 'Hapeville City', False), #4504
    (0.0, 0.0, 'Sandy Springs City', False), #4646

]

# (GRAPH APPROACH - automate creation of nodes)
'''
load_g = False
if not load_g:
    gdf = gpd.read_file(cwd / Path('data/tl_2022_us_zcta520/tl_2022_us_zcta520.shp'))
    gdf = gdf[gdf['ZCTA5CE20'] == '11206']
    shape = gdf.iloc[0].geometry
    g = ox.graph_from_polygon(shape, network_type='drive', simplify=True)
    g = g.subgraph(max(nx.strongly_connected_components(g), key=len)).copy()
    g = nx.convert_node_labels_to_integers(g)
    with open(Path(cwd / 'data/tl_2022_us_zcta520/williamsburg.pkl'), 'wb') as file:
        pickle.dump(g, file)
else:
    with open(Path(cwd / 'data/tl_2022_us_zcta520/williamsburg.pkl'), 'rb') as file:
        g = pickle.load(file)
amts = [ox.nearest_nodes(g, lon, lat) for lat, lon, _ in stations]
'''

# For now, don't consider transit amenities:
# Tuple format: (lat) float, (lon) float, (name) string, (in beltline?) boolean
amts = []


# ====================================
# SIMULATION PRE-DETERMINED PARAMETERS
# ====================================
rho_l = [1] #1, 2, 4, 8 (for each iteration) rho-house capacity
alpha_l = [0.25] #0.25, 0.75 (for each iteration) lambda-transit access vs. community value
t_max_l = [5] #5000, 10000, 15000, 20000 (for each iteration) timesteps
tau = 0.5 # inequality factor in Lorentz curve

# RUN SIMULATION?
run_experiments = True

# PLOT SIMULATION?
plot_cities = True

cty_key = 'Atlanta'


# ===============
# SIMULATION CODE
# ===============

if run_experiments:
    for rho in rho_l:
        for alpha in alpha_l:

            np.random.seed(0)

            city = City(housing, amts, rho=rho)
            #city.set_amts(amts) [graph approach to setting amenities]
            n = len(city.allNodes)
            
            agt_dows = np.diff([1 - (1 - x) ** tau for x in np.linspace(0, 1, n + 1)])
            agts = [Agent(i, dow, city, alpha=alpha) for i, dow in enumerate(agt_dows)]

            city.set_agts(agts)
            city.update()

            for t in range(max(t_max_l)):
                print('t: {0}'.format(t))
                for a in agts:
                    a.act()
                city.update()
                for a in agts:
                    a.learn()

                if t + 1 in t_max_l:

                    for a in city.agts:
                        a.avg_probabilities = a.tot_probabilities / (t + 1)

                    with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t + 1)), 'wb') as file:
                        pickle.dump(city, file)

#[GRAPH APPROACH]
#TODO: figure out how to visualize
'''
if plot_cities:
    for rho in rho_l:
        for alpha in alpha_l:
            for t_max in t_max_l:
                with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t_max)), 'rb') as file:
                    city = pickle.load(file)
                cmap = 'YlOrRd'
                figkey = '{0}_{1}_{2}_{3}'.format(cty_key, rho, alpha, t_max)
                city.plot(cmap=cmap, figkey=figkey)
                '''

c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new
t: 0
t: 1
t: 2
t: 3
t: 4


"\nif plot_cities:\n    for rho in rho_l:\n        for alpha in alpha_l:\n            for t_max in t_max_l:\n                with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t_max)), 'rb') as file:\n                    city = pickle.load(file)\n                cmap = 'YlOrRd'\n                figkey = '{0}_{1}_{2}_{3}'.format(cty_key, rho, alpha, t_max)\n                city.plot(cmap=cmap, figkey=figkey)\n                "